In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BSoup
from datetime import datetime
OK = True
BAD = False

In [5]:
def check_response_ok (response):
    import sys
    try:
        response.raise_for_status ()
        return OK
    except:
        f = open ("IBD_errors.log", "a")
        f.write ('{:%m/%d/%Y %H:%M:%S}'.format (datetime.now ()))
        f.write (" -- {}:\n\t\t{}".format (sys.exc_info ()[0], response.url))
        return BAD
    return BAD

In [25]:
def get_Ibd_rank_and_group (ticker):
    import numpy as np
    #return the result as a dict containing values for keys 'group', 'rank', 'leader'
    #if urls can not be accessed, returns an empty dict
    #if not all values are found, returns NaN for the missing values
    result = {}
    #first, search for the ticker symbol
    url_name1 = "http://myibd.investors.com/search/searchresults.aspx?Ntt={}".format (ticker.lower ())
    print ("url1: {}".format (url_name1))
    response1 = requests.get(url_name1)
    if (check_response_ok (response1)):
        #then, find the link to the stock's page by following down the chain of links
        #not sure if there is a more direct path to the stock's page
        soup1 =  BSoup (response1.content, "lxml")
        #if not a direct hit, there is a second step (hop) necessary
        url_name2 = soup1.find ("a", {"id": "ctl00_ctl00_secondaryContent_leftContent_SearchResultsMgrCtrl_didYouMeanCompanyRepeater_ctl00_didYouMeanLink"})
        print ("url2: {}".format (url_name2))
        if url_name2:
            response2 = requests.get (url_name2.get('href'))
            if (check_response_ok (response2)):
                soup2 = BSoup (response2.content, "lxml")
            #<span id="ctl00_ctl00_secondaryContent_leftContent_CustomContentCtrl_StockTwitMiniChart1_lblCompany" onclick="javascript:window.open('http://research.investors.com/stock-quotes/nyse-thor-industries-inc-tho.htm')">Thor Industries Inc</span>
                url_name3 = soup2.find ("span", {"id": "ctl00_ctl00_secondaryContent_leftContent_CustomContentCtrl_StockTwitMiniChart1_lblCompany"})
                print ("url3a: {}".format (url_name3))
            else:
                #---> what to do? return None, return NaN, set something to NaN?
                return result
        else:
            url_name3 = soup1.find ("span", {"id": "ctl00_ctl00_secondaryContent_leftContent_CustomContentCtrl_StockTwitMiniChart1_lblCompany"}) 
            print ("url3b: {}".format (url_name3))
        if not url_name3:
            return result
        tokens = url_name3.get ('onclick').split ("'")
        #---> check that tokens contains valid data
        if not tokens:
            return result
        for t in tokens:
            if "http://" in t:
                stock_link = t
                break
        response3 = requests.get (stock_link)
        if (check_response_ok (response3)):
            soup3 = BSoup (response3.content, "lxml")
            #finally got the page. Now find the ticker's group, ranking, and no.1 in that group
            market_group = soup3.find ("div", {"class": "spansubHead"})
            if (market_group):
                result["group"] = market_group.text
            else: 
                result["group"] = None

            #this span occurs only once, always contains the rank of the current ticker
            ticker_rank = soup3.find ("span", {"id":"ctl00_ctl00_secondaryContent_leftContent_GrpLeaders_ltlSymbolRank"})
            if (ticker_rank):
                result["rank"] = ticker_rank.text
            else:
                result["rank"] = np.nan

            #if ticker is not no. 1, the leader can be found in another anchor-tag inside the StockRolldiv
            #this will return None if current ticker is no. 1
            group_no1 = soup3.find ("a", {"class": "stockRoll"})
            if (group_no1):
                result["leader"] = group_no1.text
            else:
                result["leader"] = None
        else:
            return None
    
    #did not find a valid website on IBD for this ticker symbol
    return result



In [26]:
def add_Ibd_data (df):
    df["IBD_group"] = ""
    df["IBD_rank"] = 0
    for ticker in df.Symbol:
        ibd_data = get_Ibd_rank_and_group (ticker)
        print (ibd_data)
        if "group" in ibd_data:
            df["IBD_group"][df.Symbol == ticker] = ibd_data["group"]
        if "rank" in ibd_data:
            df["IBD_rank"][df.Symbol == ticker] = ibd_data["rank"]
    return df

In [6]:
def get_annual_sales_data (ticker):
    response = requests.get("https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols={}&period=annual".format (ticker))
    print (response.url)
    #now the html code is in the requests object
    #BeautifulSoup can parse html code and convert into something usable
    if check_response_ok (response):
        soup =  BSoup (response.content, "lxml")
        #<th class="top-bottom-border col-96PX lft-border" scope="col"><span class="bold">2017</span> (06/30 - 03/31) </th>
        dt=soup.find ("table", {"class": "datatable-component"})
        #the summary attribute contains a short description of the table, check that this is the Net Income table
        #print ("net income" in dt.get("summary").lower ())
        #find all rows in this table; the first row contains the header with the dates
        allrows = dt.findChildren (name = "tr")
        firstrow=(allrows[0])
        #get the dates on the table:
        cols = []
        for f in firstrow.find_all ("th"):
            cols.append(f.text[7:-2])
        alldatarows = (allrows[1:])
        idxs= ["0",]
        for row in allrows[1:]:
            f = row.find ("th")
            if f:
                idxs.append (f.text)
        new_table = pd.DataFrame(columns=cols, index = idxs) # I know the size 

        row_marker = 0
        for row in dt.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                if not "Log in" in column.get_text():
                    new_table.iat[row_marker,column_marker] = column.get_text()
                else:
                    new_table.iat[row_marker, column_marker] = 0.0
                column_marker += 1
            row_marker += 1

    return (new_table)

In [7]:
def get_quarterly_sales_data (ticker):
    response = requests.get("https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols={}&period=quarterly".format (ticker))
    print (response.url)
    #now the html code is in the requests object:
    #BeautifulSoup can parse html code and convert into something usable
    if check_response_ok (response):
        soup =  BSoup (response.content, "lxml")
        #<th class="top-bottom-border col-96PX lft-border" scope="col"><span class="bold">2017</span> (06/30 - 03/31) </th>
        dt=soup.find ("table", {"class": "datatable-component"})
        #the summary attribute contains a short description of the table, check that this is the Net Income table
        #print ("net income" in dt.get("summary").lower ())
        #find all rows in this table; the first row contains the header with the dates
        allrows = dt.findChildren (name = "tr")
        firstrow=(allrows[0])
        #get the dates on the table:
        cols = []
        for f in firstrow.find_all ("th"):
            cols.append("{}/{}".format(f.text[6:11], f.text[:4]))
        alldatarows = (allrows[1:])
        idxs= ["0",]
        for row in allrows[1:]:
            f = row.find ("th")
            if f:
                idxs.append (f.text)
        new_table = pd.DataFrame(columns=cols, index = idxs) # I know the size 

        row_marker = 0
        for row in dt.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                if not "Log in" in column.get_text():
                    new_table.iat[row_marker,column_marker] = column.get_text()
                else:
                    new_table.iat[row_marker, column_marker] = 0.0
                column_marker += 1
            row_marker += 1

    return (new_table)

In [1]:
def add_initial_sales_data (df):
    df["current_year"] = 0.0
    df["last_year"] = 0.0
    df["last_last_year"] = 0.0
    df["current_Q"] = 0.0
    df["last_Q"] = 0.0
    df["last_last_Q"] =0.0
    df["current_Y_date"] = ""
    df["current_Q_date"] = ""
    for symbol in df.Symbol:
        annual = get_annual_sales_data (symbol)
        quarterly = get_quarterly_sales_data (symbol)
        df.loc["current_year", df.Symbol == symbol] = annual.loc["Sales/Turnover (Net)"].iloc[0]
        df.loc["last_year", df.Symbol == symbol] = annual.loc["Sales/Turnover (Net)"].iloc[1]
        df.loc["last_last_year", df.Symbol == symbol] = annual.loc["Sales/Turnover (Net)"].iloc[2]
        df.loc["current_Q", df.Symbol == symbol] = quarterly.loc["Sales/Turnover (Net)"].iloc[0]
        df.loc["last_Q", df.Symbol == symbol] = quarterly.loc["Sales/Turnover (Net)"].iloc[1]
        df.loc["last_last_Q", df.Symbol == symbol] = quarterly.loc["Sales/Turnover (Net)"].iloc[2]
        df.loc["current_Y_date", df.Symbol == symbol] = annual.columns[0]
        df.loc["current_Q_date", df.Symbol == symbol] = quarterly.columns[0]
        print (df[df.Symbol == symbol])
    return df

In [ ]:
def sales_data_initial_analysis (df):\
    if not "Sales" in df.columns:
        print ("Error: no sales data found in dataframe")
        exit (1)
    

In [30]:
if __name__ == "__main__":
    df = pd.read_excel ("screener_results_IBD.xls")
    #print (df[df.Symbol == "TX"])
    #df = add_Ibd_data (df)
    df = add_initial_sales_data (df)
    df.to_excel("screener_results_IBD_sales.xls")

https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=ADBE&period=annual
https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=ADBE&period=quarterly


C:\Users\Carola\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Carola\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\Carola\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\Carola\Anaconda3\lib\site-packages\ipyk

  Symbol       Company Name Security Type  Security Price  \
0   ADBE  Adobe Systems Inc            CS          149.18   

   EPS Growth (Last Qtr vs. Same Qtr Prior Yr)  EPS Growth (TTM vs Prior TTM)  \
0                                     55.55556                        62.7551   

   Return on Equity (TTM)  Positive Earnings Surprises (90 Days)  \
0                 20.5501                                9.34394   

   Institutional Ownership (Last vs. Prior Qtr)  Net Insider Shares Bought  \
0                                        2.6803                        0.0   

      ...      Industry          Sub-Industry                     IBD_group  \
0     ...      Software  Application Software  Computer Sftwr-Desktop Group   

  IBD_rank  current_year last_year last_last_year current_Q last_Q last_last_Q  
0        2         5,854     4,796          4,147     1,841  1,772       1,682  

[1 rows x 21 columns]
https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jh

https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=AM&period=annual
https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=AM&period=quarterly
  Symbol                  Company Name Security Type  Security Price  \
7     AM  Antero Midstream Partners LP            UT           31.53   

   EPS Growth (Last Qtr vs. Same Qtr Prior Yr)  EPS Growth (TTM vs Prior TTM)  \
7                                     44.44444                           48.0   

   Return on Equity (TTM)  Positive Earnings Surprises (90 Days)  \
7                 22.8176                                    NaN   

   Institutional Ownership (Last vs. Prior Qtr)  Net Insider Shares Bought  \
7                                       -2.4342                        0.0   

      ...                         Industry  \
7     ...      Oil, Gas & Consumable Fuels   

                         Sub-Industry 

https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=ASML&period=annual
https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=ASML&period=quarterly
   Symbol     Company Name Security Type  Security Price  \
14   ASML  ASML Holding NV            DR           171.2   

    EPS Growth (Last Qtr vs. Same Qtr Prior Yr)  \
14                                     44.44444   

    EPS Growth (TTM vs Prior TTM)  Return on Equity (TTM)  \
14                           60.0                 18.8249   

    Positive Earnings Surprises (90 Days)  \
14                                    NaN   

    Institutional Ownership (Last vs. Prior Qtr)  Net Insider Shares Bought  \
14                                       -1.2863                        NaN   

       ...                                      Industry  \
14     ...      Semiconductors & Semiconductor Equipment   

           

...

https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=SITE&period=annual
https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=SITE&period=quarterly
    Symbol                  Company Name Security Type  Security Price  \
139   SITE  SiteOne Landscape Supply Inc            CS            58.1   

     EPS Growth (Last Qtr vs. Same Qtr Prior Yr)  \
139                                     57.35294   

     EPS Growth (TTM vs Prior TTM)  Return on Equity (TTM)  \
139                           50.0                 27.3711   

     Positive Earnings Surprises (90 Days)  \
139                                    NaN   

     Institutional Ownership (Last vs. Prior Qtr)  Net Insider Shares Bought  \
139                                       19.8152                        0.0   

        ...                              Industry  \
139     ...      Trading Companies & Distri

https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=TAL&period=annual
https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=TAL&period=quarterly
    Symbol         Company Name Security Type  Security Price  \
146    TAL  TAL Education Group            DR           33.71   

     EPS Growth (Last Qtr vs. Same Qtr Prior Yr)  \
146                                     99.62547   

     EPS Growth (TTM vs Prior TTM)  Return on Equity (TTM)  \
146                       30.04246                 21.0164   

     Positive Earnings Surprises (90 Days)  \
146                                    NaN   

     Institutional Ownership (Last vs. Prior Qtr)  Net Insider Shares Bought  \
146                                       -1.6355                        NaN   

        ...                           Industry        Sub-Industry  \
146     ...      Diversified Consumer Services

https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=TSS&period=annual
https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=TSS&period=quarterly
    Symbol                Company Name Security Type  Security Price  \
153    TSS  Total System Services Inc.            CS            65.5   

     EPS Growth (Last Qtr vs. Same Qtr Prior Yr)  \
153                                     63.15789   

     EPS Growth (TTM vs Prior TTM)  Return on Equity (TTM)  \
153                        4.59184                 17.7348   

     Positive Earnings Surprises (90 Days)  \
153                                5.85305   

     Institutional Ownership (Last vs. Prior Qtr)  Net Insider Shares Bought  \
153                                        3.6086                        0.0   

        ...         Industry                           Sub-Industry  \
153     ...      IT Services  D

https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=VZ&period=annual
https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=VZ&period=quarterly
    Symbol                Company Name Security Type  Security Price  \
160     VZ  Verizon Communications Inc            CS           49.49   

     EPS Growth (Last Qtr vs. Same Qtr Prior Yr)  \
160                                    529.41176   

     EPS Growth (TTM vs Prior TTM)  Return on Equity (TTM)  \
160                       10.16949                 69.5001   

     Positive Earnings Surprises (90 Days)  \
160                                    NaN   

     Institutional Ownership (Last vs. Prior Qtr)  Net Insider Shares Bought  \
160                                       -0.0295                        0.0   

        ...                                    Industry  \
160     ...      Diversified Telecommunicatio

https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=YUMC&period=annual
https://eresearch.fidelity.com/eresearch/evaluate/fundamentals/financials.jhtml?stockspage=incomestatement&symbols=YUMC&period=quarterly
    Symbol            Company Name Security Type  Security Price  \
167   YUMC  Yum China Holdings Inc            CS           39.97   

     EPS Growth (Last Qtr vs. Same Qtr Prior Yr)  \
167                                         35.0   

     EPS Growth (TTM vs Prior TTM)  Return on Equity (TTM)  \
167                       45.45455                 23.2471   

     Positive Earnings Surprises (90 Days)  \
167                                    NaN   

     Institutional Ownership (Last vs. Prior Qtr)  Net Insider Shares Bought  \
167                                        1.0126                        0.0   

        ...                           Industry Sub-Industry  \
167     ...      Hotels, Restaurants & Leisur